# 🎓 Master Thesis: Prompt Engineering with Mistral-7B-Instruct
This notebook demonstrates Prompt Engineering using the Mistral-7B-Instruct model on a dataset of beginner-level Python Q&A pairs.

We'll test zero-shot and few-shot prompting strategies and store the results for evaluation.

In [ ]:
# ✅ Install Required Libraries
!pip install -q transformers accelerate datasets huggingface_hub

In [ ]:
# ✅ Login to HuggingFace (insert your token below)
from huggingface_hub import login
login('hugging face user token')  # Replace with your actual token inside quotes

In [3]:
# ✅ Import Libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch, json
from pathlib import Path

In [ ]:
from huggingface_hub import login
login("hugging face user token")  # ← paste your real Hugging Face token here in quotes


# ✅ Load Mistral-7B-Instruct Model
model_id = 'google/gemma-2b-it'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
# ✅ Ask the model a custom question (for demo purposes)

def ask_model(question_text, max_tokens=2000):
    prompt = f"You are a helpful Python programming assistant.\n\nQuestion: {question_text}\nAnswer:"

    try:
        response = generator(prompt, max_new_tokens=max_tokens, do_sample=False)[0]['generated_text']
        print("🧠 Model Response:\n")
        print(response.split("Answer:")[1].strip())
    except Exception as e:
        print("❌ Error during generation:", e)

# 🧪 Example usage — replace the question below with your own
ask_model("What’s the official syntax for the new except* clause introduced in Python 3.14?")
